# задача
создание нейро-финансиста в торговой компании.

В его обязанности входит:
1. Заполнение журнала операции.
2. Уточнение по непольным данным.
3. Подготовка и написание аналитических записок.

# подготовка окружения

In [1]:
# # загружаем модули
# !pip install -q langchain-community langchain-openai faiss-cpu

In [2]:
# импортируем библиотеки
import requests
import os
import re
import time
import pandas as pd
import ast

from dotenv import load_dotenv
# from google.colab import userdata
from openai import OpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document

# # активируем ключ api от open ai
# os.environ['OPENAI_API_KEY'] = userdata.get('open_ai_key_lessons')
load_dotenv('.env')

True

# база данных

In [3]:
# инициализируем клиента
client = OpenAI()

## df

In [4]:
# путь для выгрузки таблиц
path_download = 'https://drive.google.com/uc?export=download&id='

In [5]:
# загружаем необходимые таблицы
# таблица массив для тестирования accounting
path_download_ok = path_download + '154MKaqin1TiBZ0icxPUUEbO9QdyltJJ_'
df_ok = pd.read_csv(path_download_ok)
df_ok

,note,date of oper,sum,account,counterparty,category,date of note,id
0,12.03.24 в магазине на Ленина сегодня выручка ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"25 ноября в магазине ""Экспресс"" собрали 52 тыс...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"5 июня 2025 выручка в ""Модный дом"" — около 104...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"18 июля 2025 с автоматов в ""Сити-Плазе"" собрал...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"3 февраля 2025 с автомата в ""Городке"" пришло п...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
95,12 мая 2024 с 2487 перекинул 50000 на карту 2...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,3 ноября 2023 ушло 80000 с 2313 на счёт 2487.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,7 июня 2025 перевёл 100000 со 1231 на расчётны...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,12.03.24 в магазине на Ленина сегодня выручка ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# таблица массив для тестирования ask
path_download_ask = path_download + '1dIwEUvlxJ6EEuw8z8SjhYGQh5MDBCeJZ'
df_ask = pd.read_csv(path_download_ask)
df_ask

,note,date of oper,sum,account,counterparty,category,date of note,id
0,"Всем привет! Закрыли смену, выручка за сегодня...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Только что продали классную кожаную куртку за ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Ребят, распродажа просто взлетела! Продали 50 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Привет, команда! Автомат на станции ""Восточная...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Ура! Автомат в бизнес-центре ""Солнечный"" побил...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
95,Зачислен купонный доход по ранее приобретенным...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,Перевела 200 000 руб. с основного расчетного с...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,Осуществила перевод 150 000 руб. с основного с...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,Перечислила 50 000 руб. с текущего счета на на...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# таблица массив для тестирования analyze
path_download_analyze = path_download + '13deuMBOzcP1y0EkOPVhv6SG3crd-y9_E'
df_analyze = pd.read_csv(path_download_analyze)
df_analyze

,note,date of oper,sum,account,counterparty,category,date of note,id
0,"Пришли, пожалуйста, аналитическую записку по д...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Подготовь, пожалуйста, записку с анализом дене...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Жду от тебя краткую записку по движению средств.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Сделай, пожалуйста, отчет по поступлениям и ра...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Нужна аналитика движения денежных средств — сд...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
95,Жду аналитическую записку по движению средств.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,Прошу прислать краткий отчет по финансовым пот...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,"Пришли, пожалуйста, аналитическую записку по д...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,"Подготовь, пожалуйста, записку с анализом дене...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# таблица для analyze
path_download_sheet = path_download + '1188mxc6_X-NOfGeDqlhbgqUk-O3U_tKn'
df_sheet = pd.read_csv(path_download_sheet)
df_sheet

,Остаток на начало месяца,"500 000,00"
0,NaN,NaN
1,1. Денежные потоки от операционной деятельности,"(605 000,00)"
2,NaN,NaN
3,1.1. Продажи через торговые точки,"1 200 000,00"
4,1.2. Продажи через вендинговые автоматы,"350 000,00"
5,1.3. Возвраты от поставщиков,"80 000,00"
6,1.4. Закупка товара,"(900 000,00)"
7,1.5. Транспортные услуги,"(120 000,00)"
8,1.6. Комиссии за эквайринг,"(45 000,00)"
9,1.7. Расчетно-кассовое обслуживание (РКО),"(15 000,00)"


In [9]:
# таблица для заполнения и итогового тестирование
path_download_total = path_download + '1lhnALWS-AL1g5fFEUXHUQEKiPJAFrCiP'
df_total = pd.read_csv(path_download_total)
df_total

,note,date,sum,account,counterparty,category,date of note,id


In [10]:
# функция для заполнения таблицы полученными значениями нейро-сотрудником
def note_to_sheet(lst, note, df):
    # индексы для ввода данных
    index_row = len(df)

    date = lst[0]
    sum = int(lst[1])
    account = int(lst[2])
    counterparty = str(lst[3])
    category =  str(lst[4])

    # записываем значения в таблицу
    df.loc[index_row] = {
        'note':note,
        'date':date,
        'sum':sum,
        'account':account,
        'counterparty':counterparty,
        'category':category
    }

    # сохранение значении в документ
    df.to_csv('test.csv', index=False)

# агенты

## accounting

In [11]:
# прописываем роль модели для заполнения таблицы, версию и температуру
system_for_accounting = '''
Ты — великолепный сотрудник финансового отдела торговой компании 'Фэмили'. Компания 
занимается реализацией одежды, игрушек и закусками.  У тебя отлично получается извлекать 
и классифицировать важные сущности из сообщений о движении денежных средств. \n\n

Пожалуйста, извлеки только 5 важных сущности из сообщения.
Ознакомся с кратим содержанием, чтобы быть в контексте.
Используй предоставленную документацию для определения статьи.
Строго придерживайся формата данных - список:
['дата', сумма, счёт, 'контрагент', 'статья'] \n\n

Ты знаешь что:
1. Денежные потоки от операционной деятельности
1.1. Продажи через торговые точки
Описание: Выручка от продажи товаров через физические магазины и киоски. 
Мы продаем и реализуем товары физическим лицам или покупателям, которые приходят 
в наши торговый точки.

1.2. Продажи через вендинговые автоматы
Описание: Выручка от автоматизированных систем продаж (вендинговых аппаратов). 
Физические лица или покупатели пользуются нашими автоматами по продажам.

1.3. Возвраты от поставщиков
Описание: Суммы, полученные от поставщиков за возвращенный товар. Полученные суммы 
от наших поставщиков за некачественный товар. 

1.4. Закупка товара
Описание: Оплата товаров для перепродажи. Покупка товаров у наших поставщиков для 
торговли через торговые точки или автоматы.

1.5. Транспортные услуги
Описание: Расходы на доставку товаров и документов. Все уплаты связанные с 
логистикой товаров от наших партнеров и поставщиков. Аренда транспорта для доставки 
товаров.

1.8. Расчётно-кассовое обслуживание (РКО)
Описание: Расходы на обслуживание расчетного счета компании. Комиссии за эквайринг.

1.9. Налоги (ЕНВД, УСН 6%, транспортный налог)
Описание: Обязательные платежи в бюджет государства.

1.10. Зарплаты и налоги(НДФЛ, ПФР) производственного персонала
Описание: Расходы на оплату труда и связанные с этим обязательства для 
производственного, рабочего персонала.

1.11. Зарплаты и налоги(НДФЛ, ПФР) коммерческого персонала
Описание: Расходы на оплату труда и связанные с этим обязательства для коммерческого 
персонала. 

1.12. Зарплаты и налоги(НДФЛ, ПФР) административного персонала
Описание: Расходы на оплату труда и связанные с этим обязательства для 
административного персонала.

1.13. Обучение персонала
Описание: Инвестиции в профессиональное развитие сотрудников. Оплата подписок на 
Skillbox и другие учебные платформы.

1.14. Расходы на персонал
Описание: Социальные и мотивационные затраты на сотрудников. Корпоративы и годовщины 
компании. Подарки сотрудницам и сотрудникам. Расходы на праздники. Аренда помещении 
для праздников. Покупка рабочей одежды и формы для персонала. Плановые медосмотры.

1.15. Командировочные расходы
Описание: Расходы, связанные с деловыми поездками сотрудников.

1.16. Представительские расходы
Описание: Расходы на поддержание деловых отношений с партнерами.

1.17. Поиск и найм персонала
Описание: Расходы на привлечение новых сотрудников. Оплата подписок на hh, SuperJob 
и другие.

1.18. Реклама и маркетинг
Описание: Расходы на продвижение бизнеса.

1.19. Содержание торговых точек и офиса
Описание: Расходы на поддержание рабочих помещений. Закупка концелярии в офис. Покупки
воды для кулеров.

1.20. Аренда офиса и магазинов
Описание: Платежи за использование недвижимости.

1.21. Покупка наличности
Описание: Комиссии за снятие наличных денег в банкоматах или банках для нужд компании. 

1.22. Прочие операционные расходы
Описание: Мелкие расходы(сумма ниже 10000), не вошедшие в другие статьи. Эта статья всегда 
выбирается если остальные не подходят.

2. Денежные потоки от инвестиционной деятельности
2.1. Покупка основных средств (ОС)
Описание: Приобретение дорогостоящего (сумма свыше 40000) имущества и оборудования для ведение 
бизнеса работающего дольше года.

2.2. Ремонт основных средств
Описание: Капитальный ремонт, увеличивающий срок службы дорогостоящего имущества. Ремонт 
дорогостоящего оборудования.

2.3. Продажа основных средств
Описание: Доходы от реализации дорогостоящего имущества отработавщего дольше года.

2.4. Выдача кредитов и займов
Описание: Деньги которые мы предоставили третьим лицам - сотрудникам, поставщикам, 
партнерам.

2.5. Возврат кредитов и займов
Описание: Поступления от погашения долгов. Деньги которые нам вернули третьи лица - 
сотрудники, сотрудницы, поставщики, партнеры.

3. Денежные потоки от финансовой деятельности
3.1. Получение кредитов и займов
Описание: Привлеченные заемные средства. Деньги которые предоставили и предоставляют 
нам банки, поставщики, партнеры.

3.2. Оплаты по кредитам, займам и овердрафту
Описание: Погашение долговых обязательств. Деньги, кредиты, займы которые мы 
возвращаем банкам, поставщикам, партнерам. 

3.3. Вклады от собственников
Описание: Деньги, вложенные учредителями в бизнес и его развитие. 

3.4. Дивиденды
Описание: Выплаты собственникам. 

3.5. Прочие поступления от финансовых операций
Описание: Доходы от финансовых активов(облигации). Проценты или дивиденды полученные от вложения 
в другие компании. Проценты по остаткам на счетах. 

Технические операции
4.1. Перевод между счетами компании
Описание: Перевод денег на другой наш счет, карту или кошелёк.
 \n\n
'''

model_for_accounting = 'gpt-4.1-mini-2025-04-14'

temperature_for_accounting = 0

In [12]:
# функция для извлечения важных сущностей из сообщении и заполнения таблицы
def accounting(system, model, temperature, note, summary, df, verbose=1):

    user_for_accounting = f'''
Давай действовать последовательно:

Шаг 1: Извлекай дату операции из сообщения. Если дата указана — добавляй её в
список в формате ДД.ММ.ГГГГ. Если дата в сообщении отсутствует — ставь «-».

Шаг 2: Извлекай сумму средств из сообщения. Если указана сумма коммисии - 
добавляй ее в список в виде положительного числа. Если сумма указана — добавляй 
её в список в виде положительного числа. Если сумма в сообщении отсутствует — 
ставь «-».

Шаг 3: Извлекай счёт или номер карты из сообщения. Если счёт указан — вноси в 
список последние четыре цифры счёта. Если счёт в сообщении отсутствует — ставь 
«-».

Шаг 4: Извлекай только наименование организации или ФИО получателя/отправителя. 
В случае с переводм средств на другой счет укажи на какой счет или номер карты 
был получен перевод. В случае с продажами через торговые точки и продаж через 
вендинговые автоматы укажи название компании 'Фэмили'. 
Если наименование организации или ФИО получателя/отправителя указано в сообщении —
вноси его в список. Если наименование контрагента или название организации
контрагента отсутствует — ставь «-».

Шаг 5: Определи финансовую статью на основании предыдущих шагов. Если удаётся 
определить статью — точно внеси её полное название(оно стоит после цифр) 
точно так, как указано в документации. Не добавляй ничего кроме названия статьи. 
Если определить статью невозможно — ставь «-».

Шаг 6. Выведи полученный список из 5 сущностей(элементов списка). \n\n

Сообщение: {note} \n
Контекст: {summary} \n\n
Ответ:
'''

    assistant = '''
['25.08.2025', 52000, 3123, 'Фэмили', 'Продажи через торговые точки']
'''
    messages = [
        {'role':'system','content':system},
        {'role':'user','content':user_for_accounting},
        {'role':'assistant', 'content':assistant}
        ]

    completion = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = temperature
    )

    answer = completion.choices[0].message.content

    if verbose:
        print('\n accounting: \n', answer)

    # обработчик ошибок
    try:
        result = ast.literal_eval(answer)
        if not isinstance(result, list):
            result = ['Error - it is not list']
    except:
        result = ['Error - accounting can not convert']

    # значения флага по умолчанию
    was_written_to_sheet = False

    # условие по определению полноты данных, если все данные получены происходит
    # запись данных в таблицу
    if result.count('-') == 0:
        #
        summary += note
        note_to_sheet(result, summary, df)
        #
        was_written_to_sheet = True

    return result, was_written_to_sheet

## ask

In [13]:
# определяем роль для модели уточнения, версию и температуру
system_for_ask = '''
Ты — великолепный сотрудник финансового отдела торговой компании 'Фэмили'. Компания 
занимается реализацией одежды, игрушек и закусками. У тебя прекрасно
получается определять нужно ли задавать уточняющие вопросы, а так же ты умеешь
их задавать. \n\n

Твоя основная задача - проследить чтобы все поля в таблице были заполнены. Тебе
будет предоставлятся список сущностей: [дата, сумма, счет, контрагент, описание]. 
Ознакомся с ним и там где «-», где отсутсутсвуют данные, задай вопросы для уточнения. \n\n

Пожалуйста, не сообщай, что заполняешь список.
'''

model_for_ask = 'gpt-4.1-nano-2025-04-14'

temperature_for_ask = 0

In [14]:
# функция для уточнения недостоющих данных
def ask(system, model, temperature, note, verbose=1):

    user_for_ask = f'''
Пожалуйста, ознакомся со списком {note}. \n\n

Давай действовать последовательно: \n
Шаг 1: Есть ли в списке «-», иначе сразу закончи диалог. \n
Шаг 2: Определяем недостающие сущности. \n
Шаг 3: Формируем вопросы для уточнения. \n
Шаг 4: Выводим только лишь вопрос(-ы) для уточнения. \n\n

Пожалуйста, задай все вопрос(-ы) в одном сообщении.
Ответ:
'''

    messages = [
        {'role':'system', 'content':system},
        {'role':'user', 'content':user_for_ask}
    ]



    completion = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = temperature
    )

    answer = completion.choices[0].message.content

    if verbose:
        print('\n ask: \n', answer)

    return answer

### assistant для тест ask

In [15]:
#
system_for_assistant = '''
Ты — умный помощник для тестирования. Твоя задача — ответить на 
вопрос(-ы)
'''


model_for_assistant = 'gpt-4.1-nano-2025-04-14'

temperature_for_assistant = 0.5

In [16]:
#
def assistant(system, model, temperature, questions, summary, verbose=0):

    user_for_assistant = f'''
Ответь на вопрос(-ы).
Если спрашивают про дату ответь 15.03.2025;
Если спрашивают про сумму ответь 50000;
Если спрашивают про счёт ответь счёт 3435;
Если спрашивают про контрагента ответь Компания В;
Если спрашивают про описание ознакомся с контекстом 
и придумай подходящее описание.

Пожалуйста, дай ответы на вопрос(-ы) в одно сообщении.

История диалога:
{summary}

Вопросы:
{questions}

Ответы:
'''

    messages = [
        {'role':'system', 'content':system},
        {'role':'user', 'content':user_for_assistant}
    ]

    completions = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = temperature
    )

    answer = completions.choices[0].message.content

    if verbose:
        print('\n assistant: \n', answer)

    return answer

## extract

In [17]:
# определяем роль модели, версию и температуру
system_for_extract = '''
Ты замечательный саммаризатор у тебя отлично получается 
анализировать тексты и веделять ключевые сущности. \n\n

Сущности для выделения:

Дата операции - указана
Сумма операции - 
Комиссия операции(если есть) - 
Счет операции - 
Название контрагента - 
Тип контрагента - 
Описание операции - 
'''

model_for_extract = 'gpt-4.1-nano-2025-04-14'

temperature_for_extract = 0

In [18]:
# функция для сохранения контекста диалога
def extract(system, model, temperature, text, verbose=1):

    user_for_extract = f'''
Пожалуйста, выдели ключевые сущности в тексте: {text}
'''

    messages = [
        {'role':'system', 'content':system},
        {'role':'user', 'content':user_for_extract}
    ]

    completion = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = temperature
    )

    answer = completion.choices[0].message.content

    if verbose:
        print('\n memory: \n', answer)

    return answer

## analyze

In [19]:
# прописываем роль, версию, температура модели
system_for_analyze = '''
Тебя зовут Пак Чхоа. Ты — великолепный сотрудник финансового отдела 
торговой компании "Фэмили". Компания занимается реализацией одежды, 
игрушек и закусками. Ты великолепный финансист в торговой компании, у тебя
изумительно получается анализировать отчеты. Пожалуйста, будь внимательной
и точной в цифрах.

Тебе предоставлен отчет компании и тебе нужно сформулировать аналитическую записку
для руководства компании. \n\n

Ты знаешь что отчеты могут быть: \n
 - ОДДС или Отчет о движении денежных средств \n
 - ББ или Бухгалтерский баланс \n
 - ОПиУ или Отчет о прибыли и убытках. \n\n

Твой ответ — только аналитическая записка, без технических пояснений.
'''

model_for_analyze = 'gpt-4.1-nano-2025-04-14'

temperature_for_analyze = 0

In [20]:
# функция для анализа отчетности
def analyze(system, model, temperature, sheet, verbose=1):

    user_for_analyze = f'''
Пожалуйста, давай действовать последовательно: \n
Шаг 1: Определи вид предоставленного отчета. \n
Шаг 2: Проанализируй отчет учитывая данные из Шаг 1. \n
Шаг 3: Напиши аналитическую записку для руководства. \n\n

Отвечай, пожалуйста, точно, и ничего не придумывай от себя.
В ответе напиши **только аналитическую записку**, как если 
бы ты писал её для руководителя. Не описывай шаги, не 
объясняй ход работы. \n\n

Предоставленный тебе отчет: {sheet}
Ответ:
'''

    messages = [
        {'role':'system', 'content':system},
        {'role':'user', 'content':user_for_analyze}
    ]

    completion = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = temperature
    )

    answer = completion.choices[0].message.content

    if verbose:
        print('\n analyze: \n', answer)

    return answer

## router

In [21]:
# определяем роль, версию и температуру модели для аналитики
system_for_router = '''
Ты — великолепный сотрудник финансового отдела торговой компании "Фэмили". 
Компания занимается реализацией одежды, игрушек и закусками. У тебя 
отлично получаеться выбирать к какой модели следует обратиться для 
эффективной работы. \n\n

Ты выбираешь одну из трех моделей: 'accounting', 'analyze', 'error'. 
Пожалуйста, при выборе моделей учитывай контекст диалога. 
\n\n

Выбирай модель по следующим правилам:

- Если сообщение — это ответ или отчет → выбери 'accounting'.
- Если сообщение — это запрос на аналитическую записку → выбери 'analyze'.
- Если сообщение бессмысленное → выбери 'error'

Важно: твой ответ должен быть только одной строкой, содержащей имя модели.
'''


model_for_router = 'gpt-4.1-nano-2025-04-14'
# model_for_router = 'gpt-4.1-mini-2025-04-14'

temperature_for_router = 0

In [22]:
# функция маршрутизатор
def router(system, model, temperature, note, summary, verbose=1):

    user_for_router = f'''
Пожалуйста, давай действовать последовательно: \n
Шаг 1: Ознакомся с контекстом диалога. \n
Шаг 2: Проанализируй сообщение сотрудника. \n
Шаг 3: На основе Шаг 1 и Шаг 2 напиши одну модель для ответа сотрудникам. \n\n

Сообщение клиента: {note} \n\n
Контекст диалога: {summary} \n\n
Ответ:
'''

    messages = [
        {'role':'system', 'content':system},
        {'role':'user', 'content':user_for_router}
    ]

    completion = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = temperature
    )

    answer = completion.choices[0].message.content

    if verbose:
        print('\n router: \n', answer)

    return answer

# нейро-сотрудник

In [23]:
# готовый нейро-сотрудник после объединения всех агентов и функции
def run_dialog(text, total_summary=''):

    # выводим полученное сообщение
    print('request: \n', text)

    if text == 'stop':
        out_answer = '\n end'
        return out_answer
    else:
        # задействуем маршрутизатор
        output = router(system_for_router, model_for_router, temperature_for_router,
                        text, total_summary)

        # условие по задействованию агентов
        if 'accounting' in output:

            out_answer, was_written = accounting(system_for_accounting,
                                                    model_for_accounting,
                                                    temperature_for_accounting, 
                                                    text, total_summary, df_total)
            total_summary += text
            # проверка значения флага очистки контекста после получения всех данных
            if was_written == True:
                total_summary = ''
                return 'stop'
            else:
                questions = ask(system_for_ask, model_for_ask, temperature_for_ask,
                                out_answer)
                total_summary += questions
                # вызов ассистента для уточнения информации и отправка ответа в роутер
                out_answer = assistant(system_for_assistant, model_for_assistant,
                                        temperature_for_assistant, questions, total_summary)
                summary = out_answer
                total_summary += summary

                return run_dialog(out_answer, total_summary)

        elif 'analyze' in output:

            out_answer = analyze(system_for_analyze, model_for_analyze,
                                    temperature_for_analyze, text)

            # вывод ответа агента - аналитика
            return out_answer
        
        elif 'error' in output:
            text = 'error text'
            out_answer = ['-', 0, 1111, '-','-']
            note_to_sheet(out_answer, text, df_total)
            
            return out_answer

        else:
            return 'error router'

# тесты

## тесты accounting

In [24]:
# функция для тестирования агента на массиве данных
def test(df, df_total):

    # перебираем в цикле заметки из массива
    for i in range(df.shape[0]):
        # извлекаем заметку и отправляем в агент accounting
        out = df.loc[i, 'note']
        result = run_dialog(out)

        print('timeout')
        time.sleep(1)

    return df_total

In [25]:
test(df_ok, df_total)

request: 
 12.03.24 в магазине на Ленина сегодня выручка была 78265, оплатили в основном картой, наличка меньше. Деньги пришли на счёт 2487, всё закрыл.

 router: 
 accounting

 accounting: 
 ['12.03.2024', 78265, '2487', 'Фэмили', 'Продажи через торговые точки']
timeout
request: 
 25 ноября в магазине "Экспресс" собрали 52 тысячи, почти всё наличкой, пару раз по QR платили. Деньги пришли на 2313.

 router: 
 accounting

 accounting: 
 ['25.11.2023', 52000, '2313', 'Фэмили', 'Продажи через торговые точки']
timeout
request: 
 5 июня 2025 выручка в "Модный дом" — около 104 тысяч. Оплаты и картами, и наличкой. Деньги зачислены на счёт 1231.

 router: 
 accounting

 accounting: 
 ['05.06.2025', 104000, '1231', 'Модный дом', 'Продажи через торговые точки']
timeout
request: 
 18 июля 2025 с автоматов в "Сити-Плазе" собрали 15200 рублей. Оплаты в основном бесконтактные, деньги прошли через 2487.

 router: 
 accounting

 accounting: 
 ['18.07.2025', 15200, '2487', 'Фэмили', 'Продажи через венд

,note,date,sum,account,counterparty,category,date of note,id
0,12.03.24 в магазине на Ленина сегодня выручка ...,12.03.2024,78265,2487,Фэмили,Продажи через торговые точки,NaN,NaN
1,"25 ноября в магазине ""Экспресс"" собрали 52 тыс...",25.11.2023,52000,2313,Фэмили,Продажи через торговые точки,NaN,NaN
2,"5 июня 2025 выручка в ""Модный дом"" — около 104...",05.06.2025,104000,1231,Модный дом,Продажи через торговые точки,NaN,NaN
3,"18 июля 2025 с автоматов в ""Сити-Плазе"" собрал...",18.07.2025,15200,2487,Фэмили,Продажи через вендинговые автоматы,NaN,NaN
4,"3 февраля 2025 с автомата в ""Городке"" пришло п...",03.02.2025,33000,2313,Фэмили,Продажи через вендинговые автоматы,NaN,NaN
...,...,...,...,...,...,...,...,...
95,12 мая 2024 с 2487 перекинул 50000 на карту 2...,12.05.2024,50000,2313,Компания В,Перевод между счетами компании,NaN,NaN
96,3 ноября 2023 ушло 80000 с 2313 на счёт 2487.К...,03.11.2023,80000,2313,Компания В,Перевод между счетами компании,NaN,NaN
97,7 июня 2025 перевёл 100000 со 1231 на расчётны...,07.06.2025,100000,2487,Компания В,Перевод между счетами компании,NaN,NaN
98,12.03.24 в магазине на Ленина сегодня выручка ...,12.03.2024,78000,2487,Фэмили,Продажи через торговые точки,NaN,NaN


In [26]:
# сохранение полученных результатов
df_total.to_csv('test_accounting.csv', index=False)

## тесты ask

In [27]:
test(df_ask, df_total)

request: 
 Всем привет! Закрыли смену, выручка за сегодня — 45к. Продали кучу всего: джинсы, свитера, обувь. Отчет и чеки в файлике, кому надо — забирайте.

 router: 
 accounting

 accounting: 
 ['-', 45000, '-', 'Фэмили', 'Продажи через торговые точки']

 ask: 
 Какая дата относится к данной операции?  
На какой счет была произведена эта транзакция?
request: 
 15.03.2025  
счёт 3435

 router: 
 accounting

 accounting: 
 ['15.03.2025', 45000, 3435, 'Фэмили', 'Продажи через торговые точки']
timeout
request: 
 Только что продали классную кожаную куртку за 12.5к (размер 46, черная). Кто еще хочет такую же — пока есть в наличии! 🚀 Чек прикрепила.

 router: 
 accounting

 accounting: 
 ['-', 12500, '-', 'Фэмили', 'Продажи через торговые точки']

 ask: 
 Какая дата относится к этой операции?  
Какой счет используется для этой транзакции?
request: 
 Дата относится к этой операции: 15.03.2025.  
Счет используется для этой транзакции: счёт 3435.

 router: 
 accounting

 accounting: 
 ['15.03.2

,note,date,sum,account,counterparty,category,date of note,id
0,12.03.24 в магазине на Ленина сегодня выручка ...,12.03.2024,78265,2487,Фэмили,Продажи через торговые точки,NaN,NaN
1,"25 ноября в магазине ""Экспресс"" собрали 52 тыс...",25.11.2023,52000,2313,Фэмили,Продажи через торговые точки,NaN,NaN
2,"5 июня 2025 выручка в ""Модный дом"" — около 104...",05.06.2025,104000,1231,Модный дом,Продажи через торговые точки,NaN,NaN
3,"18 июля 2025 с автоматов в ""Сити-Плазе"" собрал...",18.07.2025,15200,2487,Фэмили,Продажи через вендинговые автоматы,NaN,NaN
4,"3 февраля 2025 с автомата в ""Городке"" пришло п...",03.02.2025,33000,2313,Фэмили,Продажи через вендинговые автоматы,NaN,NaN
...,...,...,...,...,...,...,...,...
194,Зачислен купонный доход по ранее приобретенным...,15.03.2025,8500,3435,Компания В,Прочие поступления от финансовых операций,NaN,NaN
195,Перевела 200 000 руб. с основного расчетного с...,15.03.2025,50000,3435,депозитный счет,Перевод между счетами компании,NaN,NaN
196,Осуществила перевод 150 000 руб. с основного с...,15.03.2025,150000,3435,Компания В,Перевод между счетами компании,NaN,NaN
197,Перечислила 50 000 руб. с текущего счета на на...,15.03.2025,50000,3435,Компания В,Перевод между счетами компании,NaN,NaN


In [28]:
df_total.to_csv('test_ask.csv', index=False)

## тесты analyze

In [29]:
# тестирование агента - analyze
analyze(system_for_analyze, model_for_analyze, temperature_for_analyze, df_sheet)


 analyze: 
 В отчетности представлен Отчет о движении денежных средств (ОДДС). Анализ показывает, что за отчетный период компания увеличила остаток денежных средств на 65 000 рублей, завершив месяц с остатком 565 000 рублей. Основные источники поступлений — финансовая деятельность, в частности, привлечение кредитных средств и вклад собственников, что обеспечило приток в размере 1 100 000 рублей. В то же время, значительные расходы связаны с операционной деятельностью, включая закупку товара, оплату труда, маркетинг и аренду, что привело к отрицательному денежному потоку по операционной деятельности в размере 605 000 рублей. Инвестиционная деятельность характеризуется покупкой основных средств и выдачей кредитов, что создало отток в 430 000 рублей, однако есть поступления от продажи активов и возврата кредитов. В целом, положительный денежный поток от финансовой деятельности компенсировал убытки по операционной и инвестиционной деятельности, что позволило сохранить и увеличить денежные

'В отчетности представлен Отчет о движении денежных средств (ОДДС). Анализ показывает, что за отчетный период компания увеличила остаток денежных средств на 65 000 рублей, завершив месяц с остатком 565 000 рублей. Основные источники поступлений — финансовая деятельность, в частности, привлечение кредитных средств и вклад собственников, что обеспечило приток в размере 1 100 000 рублей. В то же время, значительные расходы связаны с операционной деятельностью, включая закупку товара, оплату труда, маркетинг и аренду, что привело к отрицательному денежному потоку по операционной деятельности в размере 605 000 рублей. Инвестиционная деятельность характеризуется покупкой основных средств и выдачей кредитов, что создало отток в 430 000 рублей, однако есть поступления от продажи активов и возврата кредитов. В целом, положительный денежный поток от финансовой деятельности компенсировал убытки по операционной и инвестиционной деятельности, что позволило сохранить и увеличить денежные средства ко

# тест сотрудника